##### import

In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time
import os.path as P
from pytorch_lightning.utilities.cloud_io import load as pl_load
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()
import matplotlib.pyplot as plt
None

In [52]:
from sources import load_dataset, default_config, parse_config
from run_train import setup_model, load_dataset
from src.trainer import Binary2DSegmentation

In [47]:
from steered_cnn.utils.rotequivariance_toolbox import *
from steered_cnn.steered_conv.steerable_filters import radial_steerable_filter, plot_filter
from steered_cnn.steered_conv import SteerableKernelBase, SteeredConv2d

In [48]:
from junno import datasets as D

### Load Experiment

In [9]:
RUN_ID = '12/5f21451fac6d45bf8ce02bdfd509a2a0'

#### Load Model

In [10]:
art_path = P.abspath(P.join('/home/gaby/mlflow/mlruns', RUN_ID, 'artifacts/'))
cfg = parse_config(art_path+'/cfg_extended.yaml')
cfg['script-arguments']['gpus']='0'

In [11]:
model = setup_model(cfg.model, 6, 1)
net = Binary2DSegmentation(model=model)
state_dict = pl_load(art_path+'/best-val-roc.ckpt'.format(25), map_location='cuda:0')['state_dict']
net.load_state_dict(state_dict)
net.eval().cuda()
None

#### Load Data

In [97]:
cfg_test = parse_config('/home/gaby/These/src/Fundus/SteeredCNN/3-Git/experimentations/EXP/steered.yaml')
train, val, test = load_dataset(cfg_test)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [95]:
print(len(train.dataset),len(val.dataset))


280 5


### Test Model

In [82]:
DRIVE = test['DRIVE'].dataset
MESSIDOR = test['MESSIDOR'].dataset

def load_sample(dataset):
    def load(id):
        sample = dataset[id]
        return sample['x'], sample['mask'], sample['y']
    return load

def run_model(x):
    if x.ndim==3:
        x=x[None]
    y = model(torch.from_numpy(x).cuda())
    return torch.sigmoid(y).detach().cpu().numpy()

messidor = D.from_numpy(id=np.arange(len(MESSIDOR))).apply({'x,mask,y':'id'}, load_sample(MESSIDOR), batchwise=False)
drive = D.from_numpy(id=np.arange(len(DRIVE))).apply({'x,mask,y':'id'}, load_sample(DRIVE), batchwise=False)

In [83]:
messidor.apply('ypred', run_model, batchwise=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DatabaseView(children=(HBox(children=(ToolBar(children=(ToolButton(html='<i class="fa fa-eye-slash" style="fon…

In [84]:
drive.apply('ypred', run_model, batchwise=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DatabaseView(children=(HBox(children=(ToolBar(children=(ToolButton(html='<i class="fa fa-eye-slash" style="fon…